# LSTM (Long Short Term Memories)
For Alnus only

## Libraries required

In [9]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import model_from_json

## Load the data

In [ ]:
pol_wu_df = read_csv('../analyze/pollen_weather.csv')
pol_wu_df = pol_wu_df[['Date','Alnus','DayOfYear','TempMax','HumidMin','VisibilityAvg']]
pol_wu_df.set_index('Date',inplace=True)
pol_wu_df.head(5)

## A function to "lag" the data

In [4]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

## "Lag" the data

In [ ]:
values = pol_wu_df.values
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# How many days to lag + how many features
n_days = 7
n_features = 4
# frame as supervised learning
reframed = series_to_supervised(scaled, n_days, 1)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[6,7,8,9]], axis=1, inplace=True)
print(reframed.head())

In [ ]:
# split into train and test sets
values = reframed.values
n_train_days = 18*365
train = values[:n_train_days, :]
test = values[n_train_days:, :]
# split into input and outputs
n_obs = n_days * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_days, n_features))
test_X = test_X.reshape((test_X.shape[0], n_days, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=100, batch_size=365, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_days*n_features))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, -4:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -4:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("LSTM_model.h5")

In [10]:
data_for_predict = read_csv('weather_today_for_LSTM.csv')
data_for_predict = data_for_predict[['Date','Alnus','DayOfYear','TempMax','HumidMin','VisibilityAvg']]
data_for_predict.set_index('Date',inplace=True)

n_days = 7
n_features = 4
n_obs = n_days * n_features

# frame as supervised learning
reframed_for_predict = series_to_supervised(data_for_predict, n_days, 1)
# drop columns we don't want to predict
reframed_for_predict.drop(reframed_for_predict.columns[[6,7,8,9]], axis=1, inplace=True)
# split into train and test sets
values_for_predict = reframed_for_predict.values
# split into input and outputs
to_predict_X, to_predict_y = values_for_predict[:, :n_obs], values_for_predict[:, -n_features]
# reshape input to be 3D [samples, timesteps, features]
to_predict_X = to_predict_X.reshape((to_predict_X.shape[0], n_days, n_features))

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("LSTM_model.h5")

# make a prediction
result = loaded_model.predict(to_predict_X)

In [11]:
list(result[0])[0]

5.451121